# DB 테이블 만들기

### 1. patent  
- patent_assignee 합친 거 만들기
- assignee 쪼개기: patent_id 지우고 drop_duplicates (pdpass로 연결하면 되니까)
- patent class, patent info 에서 중복 제거
- year 추가

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

df = pd.read_csv('fung/pid_assignee.csv')
list(df)

['patent_id',
 'assignee',
 'raw_assignee',
 'pdpass',
 'country_code',
 'state_code']

In [2]:
len(df)

2842685

__patent_assignee 만들기__

In [4]:
df1 = df.loc[:,['patent_id','assignee']]
df1.drop_duplicates(inplace=True)
len(df1)

2828844

In [6]:
type(df1.iloc[1,0])

numpy.int64

In [9]:
df1 = df1.loc[df1['patent_id'] > 6836898]
len(df1)

2070579

In [10]:
df1 = df1.loc[df1['patent_id'] < 8400000]
len(df1)

1489783

In [12]:
df1.sort_values(by='patent_id',inplace=True)
df1 = df1.loc[pd.isnull(df['assignee'])==False]
df1.to_csv('db_table/patent_assignee.csv',index=False)

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')
df1 = pd.read_csv('db_table/patent_assignee.csv')
len(df1)

1489783

In [3]:
df1 = df1.loc[pd.isnull(df1['assignee'])==False]
len(df1)

1342649

In [4]:
df1.to_csv('db_table/patent_assignee.csv',index=False)

__assignee 만들기__

In [3]:
df.drop(['patent_id','assignee'], axis=1, inplace=True)

df.drop_duplicates(inplace=True)
len(df)

336480

In [4]:
df.to_csv('assignee.csv',index=False)

In [5]:
df = pd.read_csv('patent_info.csv')
df1 = pd.read_csv('patent_class_BIO.csv')
df2 = pd.read_csv('patent_class_EE&IT.csv')

df.drop_duplicates(inplace=True)
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

df.to_csv('db_table/patent_info.csv',index=False)
df1.to_csv('db_table/patent_class_BIO.csv',index=False)
df2.to_csv('db_table/patent_class_EE&IT.csv',index=False)

In [14]:
# pdpass에서 assignee가 primary key인지 확인
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

In [16]:
pdp = pd.read_csv('db_table/pdpass.csv')
len(pdp)

247567

In [18]:
len(pdp['assignee'].unique())

247102

In [19]:
dup = pdp.duplicated('assignee')
type(dup)

pandas.core.series.Series

In [21]:
for i in range(0,len(dup)):
    if dup[i] == True:
        print i
        break

11124


In [22]:
pdp.iloc[11124,:]

pdpass                        -18181
assignee    KABUSHIKI KAISHA TOSHIBA
Name: 11124, dtype: object

__year 추가__

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

In [8]:
df=pd.read_csv('patent_class_EE&IT.csv')
def year(date):
    return date[:4]

df['year'] = df['disp_dt'].apply(year)
del df['disp_dt']
df.to_csv('patent_class_EE&IT.csv',index=False)

### 2. application
- app의 key = number로 통일

In [6]:
df = pd.read_csv('app/app_info.csv')
list(df)

['number', 'app_id', 'date', 'raw_assignee', 'country', 'granted_dum']

In [7]:
df.drop(['app_id','country'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

df.to_csv('db_table/app_info.csv',index=False)

### 3. compustat
- 바이오, 전기전자 합쳐서 큰 compu_gvkey 만들기
- 05-07, 08-12 합치기
- sic 붙이기

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

In [2]:
df = pd.read_csv('compustat_gvkey_temp.csv')
df1 = df.groupby(['gvkey','conm'], as_index=False).sum()
print len(df), len(df1)

782 743


In [4]:
df1.to_csv('db_table/compustat_gvkey_numpat.csv',index=False)

__compu 합치기__

In [6]:
os.chdir('E:/apps/compustat')
df = pd.read_csv('compustat_05-07.csv')
df1 = pd.read_csv('compustat_08-12.csv')

print len(df), len(df1)

111933 215711


In [7]:
glb = pd.concat([df,df1])

print len(df), len(df1), len(glb)

111933 215711 327644


In [8]:
glb.sort_values(by=['gvkey','fyear'], inplace=True)
glb.to_csv('compustat_05-12.csv',index=False)

__sic 붙이기__

In [5]:
os.chdir('E:/apps/compustat')
df = pd.read_csv('compustat_05-12.csv')
sic = pd.read_csv('sic.csv')
sic.drop_duplicates(inplace=True)

print len(df), len(sic)

327644 337408


In [6]:
df = df.merge(sic, how='left', on=['gvkey','fyear'])
len(df)

327644

In [7]:
df.to_csv('compustat_05-12.csv',index=False)

In [8]:
del df
del sic['fyear']

sic.drop_duplicates(inplace=True)
len(sic)

54527

In [10]:
sic.to_csv('sic_real.csv',index=False)

os.chdir('E:/apps/db_table')

bio = pd.read_csv('compustat_gvkey_numpat_BIO.csv')
ee = pd.read_csv('compustat_gvkey_numpat_EE&IT.csv')

In [11]:
print len(bio), len(ee)

299 423


In [12]:
bio = bio.merge(sic, how='left', on='gvkey')
ee = ee.merge(sic, how='left', on='gvkey')

print len(bio), len(ee)

299 423


In [14]:
bio.to_csv('compustat_gvkey_BIO.csv',index=False)
ee.to_csv('compustat_gvkey_EE&IT.csv',index=False)